In [36]:
#used https://www.youtube.com/watch?v=r9QjkdSJZ2g&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=2 
#natual language processing(tokenization)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer#tokenizer apis

vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

sentences = ['I love playing outside','I love my family', 'You love my dog!',"Do you think that I should learn how to use sckit?"]#sentences are set as arrays

#slicing up the data
training_sentences = sentences[0:training_size]
testing_sentences=sentences[training_size:]
training_labels=labels[0:training_size]
testing_labels=labels[training_size:]

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)#num words perameter, keeps most frequent 100 words
tokenizer.fit_on_texts(training_sentences)#asking the tokenizer to work it self through the code
word_index = tokenizer.word_index

print(word_index)#prints a dictionary

{'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'you': 5, 'playing': 6, 'outside': 7, 'family': 8, 'dog': 9, 'do': 10, 'think': 11, 'that': 12, 'should': 13, 'learn': 14, 'how': 15, 'to': 16, 'use': 17, 'sckit': 18}


In [37]:
sequences = tokenizer.texts_to_sequences(sentences) #text to sequences tokens
print("\nWord Index = ", word_index)
print("\nSequences = ", sequences)
from tensorflow.keras.preprocessing.sequence import pad_sequences
#padding instead of a ragged tensor
padded = pad_sequences(sequences, padding='post')

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


testing_sequences= tokenizer.texts_to_sequences(testing_sentences)
testing_padded= pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(padded.shape)
print(padded)

import json 

!wget --no-check-certificate \https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json

with open('sarcasm.json', 'r') as f:
  datastore = json.load(f)
  sentences=[]#attributes
  labels=[]
  urls=[]
  for item in datastore:
    sentences.append(item['headline'])#iterations
    labels.append(item['is_sarcastic'])
    urls.append(item["article_link"])

[[2, 3, 6, 7], [2, 3, 4, 8], [5, 3, 4, 9], [10, 5, 11, 12, 2, 13, 14, 15, 16, 17, 18]]
(4, 11)
[[ 2  3  6  7  0  0  0  0  0  0  0]
 [ 2  3  4  8  0  0  0  0  0  0  0]
 [ 5  3  4  9  0  0  0  0  0  0  0]
 [10  5 11 12  2 13 14 15 16 17 18]]
--2021-07-22 12:54:40--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 142.250.81.208, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json.4’

sarcasm.json.4      100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2021-07-22 12:54:40 (208 MB/s) - ‘sarcasm.json.4’ saved [5643545/5643545]



In [46]:
#embedding by summing up the vectors
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                         
])

In [48]:
model.compile(loss='binary_crossentrophy', optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_5 ( (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 24)                408       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [60]:
import pandas as pd
import numpy as np

datastore[0:10]
df = pd.DataFrame(datastore)
df.head(10)

#testing_sentences=sentences[training_size:]
#training_labels=labels[0:training_size]
#testing_labels=labels[training_size:]

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
5,https://www.huffingtonpost.com/entry/advancing...,advancing the world's women,0
6,https://www.huffingtonpost.com/entry/how-meat-...,the fascinating case for eating lab-grown meat,0
7,https://www.huffingtonpost.com/entry/boxed-col...,"this ceo will send your kids to school, if you...",0
8,https://politics.theonion.com/top-snake-handle...,top snake handler leaves sinking huckabee camp...,1
9,https://www.huffingtonpost.com/entry/fridays-m...,friday's morning email: inside trump's presser...,0


In [64]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [71]:
#training

num_epochs=30
history = model.fit(training_padded, 
                    training_labels, 
                    epochs = num_epochs, 
                    validation_data = (testing_padded, testing_labels),
                    verbose = 2)

import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, "accuracy")
plot_graphs(history, "loss")


ValueError: ignored

NameError: ignored

In [53]:
sentence = [
            "granny starting to fear spiders in the garden might be real", 
            "the weather today is bright and sunny",
            "sure, you definentily like ice cream",
            "wow, its so, ugh, cute."
            ]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

[[0.4966039 ]
 [0.49665508]
 [0.49672544]
 [0.49669138]]
